In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy.random import seed

from tensorflow import set_random_seed

from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
%matplotlib inline

In [3]:
seed(639)
set_random_seed(5944)

In [4]:

print(os.listdir("../input"))

['unarch', 'lanl-features.zip', 'test.zip', 'sample_submission.csv', 'train.csv.zip', 'test', 'train']


In [7]:
float_data = pd.read_csv("../input/train/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values

In [8]:
def extract_features(z):
     return np.c_[z.mean(axis=1), 
                  np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=1)),
                  z.std(axis=1)]

In [23]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
    
    print("\n")
    print("createX, x.shape:", x.shape)
    print("createX, last_index:", last_index)
    print("createX, n_steps:", n_steps)
    print("createX, step_length:", step_length)
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1).astype(np.float32) - 5 ) / 3
    temp = x[(last_index - n_steps * step_length):last_index]
    print("createX, temp.shape before reshape:", temp.shape)
    temp = temp.reshape(n_steps, -1).astype(np.float32)
    print("create_X, temp.shape after reshape:", temp.shape)
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations.
    print("createX, extract_features(temp).shape:", extract_features(temp).shape)
    print("createX, extract_features(temp[:, -step_length // 10:]).shape", extract_features(temp[:, -step_length // 10:]).shape)
    print("createX, extract_features(temp[:, -step_length // 100:]).shape", extract_features(temp[:, -step_length // 100:]).shape)
    print()
    result = np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]
    print("createX, result shape:", result.shape)
    print()
    '''
    return np.c_[extract_features(temp),
                 extract_features(temp[:, -step_length // 10:]),
                 extract_features(temp[:, -step_length // 100:])]
    '''
    return result

In [25]:
# Query "create_X" to figure out the number of features
n_features = create_X(float_data[0:150000]).shape[1]
print("Our RNN is based on %i features"% n_features)
    
# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150, step_length=1000):
    if max_index is None:
        max_index = len(data) - 1
    print("\n")   
    print("generator, data.shape:", data.shape)
    while True:
        # Pick indices of ending positions
        rows = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)
        print("generator, type(rows):", type(rows))
        print("generator, rows.shape:", rows.shape)
        print("generator rows:\n", rows)
         
        # Initialize feature matrices and targets
        samples = np.zeros((batch_size, n_steps, n_features))
        targets = np.zeros(batch_size, )
        print("generator, samples.shape:", samples.shape)
        print("generator, targets.shape:", targets.shape)
        
        for j, row in enumerate(rows):
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            print("generator, samples[{}].shape:".format(j), samples[j].shape)
            print("generator, targets[{}].shape:".format(j), targets[j].shape)
            targets[j] = data[row - 1, 1]
        yield samples, targets
        
batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
second_earthquake = 50085877
float_data[second_earthquake, 1]

# Initialize generators
# train_gen = generator(float_data, batch_size=batch_size) # Use this for better score
train_gen = generator(float_data, batch_size=batch_size, min_index=second_earthquake + 1)
valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)



createX, x.shape: (150000, 2)
createX, last_index: 150000
createX, n_steps: 150
createX, step_length: 1000
createX, temp.shape before reshape: (150000, 2)
create_X, temp.shape after reshape: (150, 2000)
createX, extract_features(temp).shape: (150, 6)
createX, result shape: (150, 18)

Our RNN is based on 18 features


In [ ]:
cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

model = Sequential()
model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.summary()

In [ ]:
model.compile(optimizer=adam(lr=0.0005), loss="mae")

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=0,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=200)

In [ ]:
def perf_plot(history, what = 'loss'):
    x = history.history[what]
    val_x = history.history['val_' + what]
    epochs = np.asarray(history.epoch) + 1
    
    plt.plot(epochs, x, 'bo', label = "Training " + what)
    plt.plot(epochs, val_x, 'b', label = "Validation " + what)
    plt.title("Training and validation " + what)
    plt.xlabel("Epochs")
    plt.legend()
    plt.show()
    return None

perf_plot(history)

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

In [ ]:
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

In [ ]:
submission.to_csv('submission.csv')

In [5]:
pd.read_csv?

In [22]:
train_gen.__next__()



generator, data.shape: (629145480, 2)
generator, type(rows): <class 'numpy.ndarray'>
generator rows:
 [392570814 210731521 566541212 224990896 476635094 112349588 483488016
 192297619 482480504  51409206 283556385 504974604 342742569 429154662
 464834154 355934085 301234308 152645226 267913021 479211806 313349275
 422203958 332023803 603258677 106770180 206792031 373950964 344025405
 567533000 556765122 237164450 516302218]
generator, samples.shape: (32, 150, 18)
generator, targets.shape: (32,)


createX, x.shape: (629145480,)
createX, last_index: 392570814
createX, n_steps: 150
createX, step_length: 392570814
createX, temp.shape before reshape: (150000,)
create_X, temp.shape after reshape: (150, 1000)
createX, extract_features(temp).shape: (150, 6)
createX, result shape: (150, 18)
generator, samples[0].shape: (150, 18)
generator, targets[0].shape: ()


createX, x.shape: (629145480,)
createX, last_index: 210731521
createX, n_steps: 150
createX, step_length: 210731521
createX, temp.sh

createX, extract_features(temp).shape: (150, 6)
createX, result shape: (150, 18)
generator, samples[21].shape: (150, 18)
generator, targets[21].shape: ()


createX, x.shape: (629145480,)
createX, last_index: 332023803
createX, n_steps: 150
createX, step_length: 332023803
createX, temp.shape before reshape: (150000,)
create_X, temp.shape after reshape: (150, 1000)
createX, extract_features(temp).shape: (150, 6)
createX, result shape: (150, 18)
generator, samples[22].shape: (150, 18)
generator, targets[22].shape: ()


createX, x.shape: (629145480,)
createX, last_index: 603258677
createX, n_steps: 150
createX, step_length: 603258677
createX, temp.shape before reshape: (150000,)
create_X, temp.shape after reshape: (150, 1000)
createX, extract_features(temp).shape: (150, 6)
createX, result shape: (150, 18)
generator, samples[23].shape: (150, 18)
generator, targets[23].shape: ()


createX, x.shape: (629145480,)
createX, last_index: 106770180
createX, n_steps: 150
createX, step_length: 106770

(array([[[ 4.97200012,  0.        ,  6.        , ..., 12.5       ,
          18.        ,  3.89999986],
         [ 4.70599985,  0.        ,  5.        , ...,  5.5       ,
           8.        ,  2.82134724],
         [ 4.67399979,  0.        ,  5.        , ...,  4.        ,
           6.        ,  1.6248076 ],
         ...,
         [ 4.69199991,  0.        ,  5.        , ...,  6.        ,
           9.        ,  2.44131112],
         [ 4.56500006,  0.        ,  5.        , ...,  8.        ,
          12.        ,  4.27083111],
         [ 4.47399998,  0.        ,  4.        , ...,  6.5       ,
          11.        ,  3.17017365]],
 
        [[ 4.41200018,  0.        ,  4.        , ...,  5.75      ,
           8.        ,  2.08806133],
         [ 4.41499996,  0.        ,  4.        , ...,  5.        ,
           5.        ,  2.11896205],
         [ 4.42399979,  0.        ,  4.        , ...,  6.        ,
           7.        ,  1.28062487],
         ...,
         [ 4.25400019,  0.       